Random Forest

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , RepeatedKFold ,cross_val_score , ParameterGrid
from sklearn.tree import DecisionTreeRegressor , export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_boston
#Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
%matplotlib inline
#metricas
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from funpymodeling.exploratory import freq_tbl , profiling_num
from sklearn import preprocessing

In [24]:
boston = load_boston()
df = np.column_stack((boston.data , boston.target))
df = pd.DataFrame(df, columns = np.append(boston.feature_names , "MEDV")) #unimos todos los datos en un unico arreglo
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


Division de los datos para entrenamiento y test

In [34]:
x = df.drop(["MEDV"] , axis = 1) #generamos la matriz de datos sin considerar las etiquetas
y = df["MEDV"] #generamos un vector de etiquetas

x_train, x_test, y_train , y_test = train_test_split (x ,y , test_size = 0.20 , random_state=42) 

Creacion y ajuste del modelo

In [35]:
#Creacion del modelo

rf_model = RandomForestRegressor(
                                n_estimators=25,
                                criterion = "mse",
                                max_depth = None,
                                max_features = "auto",
                                oob_score = False,
                                n_jobs = -1 ,
                                random_state = 123
)
rf_model.fit(x_train,y_train)

RandomForestRegressor(n_estimators=25, n_jobs=-1, random_state=123)

evaluamos su capacidad descriptiva

In [41]:
prediction = rf_model.predict(x_test)
mse = mean_squared_error(y_test,prediction)
mae = mean_absolute_error(y_test,prediction)
r2 = r2_score(y_test,prediction)

print(f"El error de (mae) en la prueba es de: {mae}")
print(f"El error de (mse) en la prueba es de: {mse}")
print(f"El error de (R2) en la prueba es de: {r2}")


El error de (mae) en la prueba es de: 2.155607843137255
El error de (mse) en la prueba es de: 9.123005803921574
El error de (R2) en la prueba es de: 0.8755961384949038


Importancia de los predictores

De forma similar al ejercicio de arboles de desicion, tambien es posible obtener la importancia de cada varibale, pero esta vez para la regresion.

In [46]:
model_var = ExtraTreesRegressor()#se inicia el modelo
model_var.fit(x_train,y_train)#se entrena el modelo

importancia_predictores = pd.DataFrame(
                                        {"Caracteristicas" : x.columns,
                                        "Importancia" : model_var.feature_importances_}
                                        )
print("Importancia de las caracteristicas en el modelo")

importancia_predictores.sort_values("Importancia" , ascending = False)

Importancia de las caracteristicas en el modelo


,Caracteristicas,Importancia
5,RM,0.359778
12,LSTAT,0.329258
10,PTRATIO,0.050967
9,TAX,0.049058
4,NOX,0.038334
0,CRIM,0.029273
7,DIS,0.028614
2,INDUS,0.028273
3,CHAS,0.021139
8,RAD,0.021071
